In [105]:
"""
Get all the match data for a certain season
"""

import requests
from bs4 import BeautifulSoup
import os
import csv
import pandas as pd

season = 4
year = season + 1956 - 2

teams = 18
if year >= 1962 and year <= 1965:
    teams = 16


season_id = str(season) if season >= 10 else "0" + str(season)

URL = "https://www.voetbalstats.nl/listjaarere.php?seizoenid=" + season_id
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

td_elements = soup.find_all("td", class_="linkere")

thuis_teams = []
uit_teams = []
data = []
uitslagen = []

for i in range(len(td_elements)):
    a_tag = td_elements[i].find("a", href=True)

    if a_tag and a_tag["href"].startswith("listclubjaarere"):
        if i > teams * 2 and len(thuis_teams) < teams * (teams - 1) and i % 2 != 0:
            thuis_teams.append(a_tag.get_text().strip())
        if i > teams * 2 and len(uit_teams) < teams * (teams - 1) and i % 2 == 0:
            uit_teams.append(a_tag.get_text().strip())

    if a_tag and a_tag["href"].startswith("opstelere"):
        if len(uitslagen) < teams * (teams - 1) and i % 2 != 0:
            uitslagen.append(a_tag.get_text().strip())
        if len(data) < teams * (teams - 1) and i % 2 == 0:
            data.append(a_tag.get_text().strip())


if uitslagen[0].count("-") == 2:
    uitslagen, data = data, uitslagen
    thuis_teams, uit_teams = uit_teams, thuis_teams


In [106]:
uitslagen

['2 - 2',
 '3 - 1',
 '0 - 1',
 '0 - 1',
 '0 - 2',
 '2 - 2',
 '1 - 0',
 '0 - 0',
 '1 - 1',
 '4 - 2',
 '2 - 3',
 '2 - 6',
 '5 - 4',
 '7 - 3',
 '0 - 0',
 '3 - 2',
 '2 - 0',
 '3 - 3',
 '2 - 2',
 '2 - 0',
 '7 - 1',
 '5 - 1',
 '2 - 2',
 '1 - 0',
 '3 - 2',
 '5 - 1',
 '3 - 3',
 '0 - 3',
 '2 - 2',
 '1 - 1',
 '2 - 0',
 '3 - 1',
 '2 - 1',
 '6 - 1',
 '3 - 1',
 '3 - 0',
 '2 - 0',
 '2 - 1',
 '4 - 2',
 '0 - 0',
 '3 - 1',
 '2 - 0',
 '4 - 1',
 '2 - 1',
 '1 - 3',
 '1 - 2',
 '5 - 3',
 '6 - 3',
 '2 - 0',
 '1 - 2',
 '0 - 0',
 '1 - 1',
 '2 - 2',
 '0 - 3',
 '3 - 3',
 '5 - 0',
 '1 - 3',
 '1 - 3',
 '6 - 0',
 '6 - 1',
 '0 - 2',
 '3 - 2',
 '2 - 1',
 '5 - 1',
 '0 - 1',
 '1 - 2',
 '3 - 6',
 '2 - 0',
 '0 - 3',
 '2 - 1',
 '3 - 3',
 '0 - 1',
 '6 - 0',
 '6 - 1',
 '1 - 4',
 '0 - 3',
 '3 - 2',
 '2 - 2',
 '1 - 0',
 '1 - 2',
 '2 - 2',
 '2 - 0',
 '3 - 1',
 '3 - 1',
 '1 - 2',
 '4 - 0',
 '3 - 0',
 '1 - 1',
 '4 - 2',
 '3 - 3',
 '6 - 2',
 '0 - 0',
 '1 - 1',
 '2 - 1',
 '2 - 2',
 '0 - 0',
 '1 - 1',
 '2 - 2',
 '2 - 0',
 '1 - 0',


In [96]:
if len(thuis_teams) != teams * (teams - 1) or len(uit_teams) != teams * (teams - 1) or len(data) != teams * (teams - 1) or len(uitslagen) != teams * (teams - 1):
    print(year)
    print("Something went wrong")
rows = []

for i in range(teams * (teams - 1)):
    thuis_doelpunten = int(uitslagen[i].split()[0])
    uit_doelpunten = int(uitslagen[i].split()[2])
    if thuis_doelpunten > uit_doelpunten:
        uitslag = "Gewonnen"
    elif thuis_doelpunten == uit_doelpunten:
        uitslag = "Gelijk"
    else:
        uitslag = "Verloren"

    rows.append([data[i], thuis_teams[i], uit_teams[i], thuis_doelpunten, uit_doelpunten, uitslag])

ValueError: invalid literal for int() with base 10: '24-08-1958'

In [84]:
df = pd.DataFrame(rows, columns=["Datum", "Thuis", "Uit", "Thuis Doelpunten", "Uit Doelpunten", "Uitslag"])

In [86]:
# Define the file path
file_path = "../processed data/" + str(year) + "/uitslagen" + str(year) + ".csv"

# Ensure the directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)

df.to_csv(file_path)

In [76]:
print(df.loc[df["Uitslag"] == "Verloren"]["Datum"].count())
print(df.loc[df["Uitslag"] == "Gewonnen"]["Datum"].count())
print(df.loc[df["Uitslag"] == "Gelijk"]["Datum"].count())

103
143
60


In [93]:
page.content

b'\xef\xbb\xbf\xef\xbb\xbf\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<HTML>\r\n<HEAD> \r\n\r\n\r\n\r\n<META NAME="author" CONTENT="Website: voetbalstats.nl - Rob Regeer"> \r\n<META HTTP-EQUIV="Pragma" CONTENT="no-cache"> \r\n<META NAME="GENERATOR" CONTENT="Mozilla/3.0Gold (Win95; I) [Netscape]"> \r\n<META NAME="DESCRIPTION" CONTENT=" Overzicht Eredivisie. De uitslagen, de spelers, topscorers, coaches, scheidsrechters en stadions in het seizoen 1957/1958.  "> \r\n<META NAME="KEYWORDS" CONTENT="1957/1958 ,Aantal, Eredivisie, wedstrijden, duels, totaal, topscorers, Nederlandse, competitie,stadions, coahes, scheidsrechters."> \r\n<META NAME="ROBOTS" CONTENT="INDEX, FOLLOW"> \r\n<META NAME="REVISIT-AFTER" CONTENT="1 MONTH"> \r\n<TITLE>Eredivisie -  seizoen 1957/1958 - Voetbalstats.nl  </TITLE> \r\n\r\n<meta property="og:title" content="Eredivisie -  seizoen 1957/1958 - Voetbalstats.nl   " />\r\n<meta property="og:type" content="voetbal" />\r\n<meta property="og:url" content="" />\r\n<meta property="og

In [2]:
"""
Get all the match data for a certain season
"""

import requests
from bs4 import BeautifulSoup
import os
import csv
import pandas as pd

for season in range(2, 5):

    year = season + 1956 - 2

    if year == 2019:
        continue

    teams = 18
    if year >= 1962 and year <= 1965:
        teams = 16

    season_id = str(season) if season >= 10 else "0" + str(season)

    URL = "https://www.voetbalstats.nl/listjaarere.php?seizoenid=" + season_id
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, "html.parser")

    td_elements = soup.find_all("td", class_="linkere")

    thuis_teams = []
    uit_teams = []
    data = []
    uitslagen = []

    for i in range(len(td_elements)):
        a_tag = td_elements[i].find("a", href=True)

        if a_tag and a_tag["href"].startswith("listclubjaarere"):
            if i > teams * 2 and len(thuis_teams) < teams * (teams - 1) and i % 2 != 0:
                thuis_teams.append(a_tag.get_text().strip())
            if i > teams * 2 and len(uit_teams) < teams * (teams - 1) and i % 2 == 0:
                uit_teams.append(a_tag.get_text().strip())

        if a_tag and a_tag["href"].startswith("opstelere"):
            if len(uitslagen) < teams * (teams - 1) and i % 2 != 0:
                uitslagen.append(a_tag.get_text().strip())
            if len(data) < teams * (teams - 1) and i % 2 == 0:
                data.append(a_tag.get_text().strip())

    if uitslagen[0].count("-") == 2:
        uitslagen, data = data, uitslagen
        thuis_teams, uit_teams = uit_teams, thuis_teams

    if len(thuis_teams) != teams * (teams - 1) or len(uit_teams) != teams * (teams - 1) or len(data) != teams * (teams - 1) or len(uitslagen) != teams * (teams - 1):
        print(year)
        print("Something went wrong")

    rows = []

    for i in range(teams * (teams - 1)):
        thuis_doelpunten = int(uitslagen[i].split()[0])
        uit_doelpunten = int(uitslagen[i].split()[2])
        if thuis_doelpunten > uit_doelpunten:
            uitslag = "Gewonnen"
        elif thuis_doelpunten == uit_doelpunten:
            uitslag = "Gelijk"
        else:
            uitslag = "Verloren"

        rows.append([data[i], thuis_teams[i], uit_teams[i], thuis_doelpunten, uit_doelpunten, uitslag])


    df = pd.DataFrame(rows, columns=["Datum", "Thuis", "Uit", "Thuis Doelpunten", "Uit Doelpunten", "Uitslag"])

    # Define paths
    raw_dir = f"../raw data/{year}"
    processed_dir = f"../processed data/{year}"
    file_path_raw = f"{raw_dir}/uitslagen{year}.txt"
    file_path_processed = f"{processed_dir}/uitslagen{year}.csv"

    # Ensure directories exist
    os.makedirs(raw_dir, exist_ok=True)
    os.makedirs(processed_dir, exist_ok=True)

    # Check and resolve conflicts for processed file
    if os.path.isfile(file_path_processed):
        os.remove(file_path_processed)  # Remove the conflicting file
    elif os.path.isdir(file_path_processed):
        import shutil
        shutil.rmtree(file_path_processed)  # Remove the conflicting directory

    # Write raw file
    with open(file_path_raw, "w", encoding="utf-8") as file:
        file.write(page.text)

    # Write processed CSV file
    df.to_csv(file_path_processed, index=False)

    print(year, "Succes")



1956 Succes
1957 Succes
1958 Succes
